# 뱅뱅막국수

In [12]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EB%B1%85%EB%B1%85%EB%A7%89%EA%B5%AD%EC%88%98/place/33964233?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            try:
                # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[5]/a[1]'
                review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[4]/a'
                review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                # 리뷰 텍스트와 날짜를 사진 여부에 따라 가져오기
                try:
                    # 사진이 있을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                except NoSuchElementException:
                    # 사진이 없을 때
                    review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                    review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_text_element)
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review_date_element)

                review_text = review_text_element.text.strip()
                review_date = review_date_element.text.strip()

                if review_text:
                    reviews.append((review_date, review_text))
                    review_count += 1
                    print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                else:
                    print(f"리뷰 {review_count}: 내용 없음")
                    review_count += 1

            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음")
                continue

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


가게 이름: 뱅뱅막국수
업종 카테고리: 국수
별점: 4.69
방문자 리뷰: 3,885
블로그 리뷰: 1,958
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 3017
리뷰 1의 '더보기' 버튼 클릭 완료
리뷰 1: 그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면발이 탱글탱글 퀄리티가 남달라요💛 포크도 뱅글뱅글모양 계란도 뱅글뱅글 가게안에 뱅글뱅글 도는 노래만 나오고 ㅋㅋ 컨셉에 진심이에요 노래도 넘 재밌어요!! 막국수 재료도 다 국내산 좋은 것만 쓰시고 원산지증명서까지 붙여놓으셨더라구용 자부심이 느껴졌습니다 참기름막국수랑 코다리막국수 먹었는데 둘다 너무너무 맛있어요 코다리회좋아해서 추가해먹었어요 >_< 인기많은 맛집엔 이유가 있어요!! 김치만두도 야들야들하고 감태추가해서 막국수 싸먹었는데 존맛이에용 감태랑 꼭 같이 드셔요 양도 많고 가게 위생도 신경쓰셔서 깨끗해요!! 점심시간에는 웨이팅 주의! 강남 가성비맛집으로 추천드려욧 (2024년 11월 7일 목요일)

리뷰 2의 '더보기' 버튼 클릭 완료
리뷰 2: 도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛집인정.참기름막국수.들기름막국수.비빔막국수 골고루 주문했어요. 진짜 양도 푸짐하고 감칠맛나고 맛있었답니다. (2024년 11월 4일 월요일)

리뷰 3의 '더보기' 버튼 클릭 완료
리뷰 3: 처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고
양도 진짜 많고!! 맛도 있었어요 👍👍

참기름 먹다가 들기름 먹으면 좀 심심하다고 느낄 수 있을 것 같은데 취향대로 드시면 될 것 같아요
만두랑 육전까지 다 맛있었어요
또오고싶네요 (2024년 11월 3일 일요일)

리뷰 4의 '더보기' 버튼 클릭 완료
리뷰 4: 뱅뱅막국수!
강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 말은 예쁜 달걀 지단 얹은 매콤 코다리 듬뿍 막국수가 정말 맛있네요.함께 나온 굴전도 넘 맛있어요.매장도 깔끔 식사하기 좋

KeyboardInterrupt: 